In [29]:
import torch
import numpy as np

In [30]:
torch.cuda.is_available()

False

In [31]:
torch.cuda.device_count()

0

In [32]:
# Convert decimal to binary string
def sources_and_subsets_nodes(N):
  str1 = "{0:{fill}"+str(N)+"b}"
  a = []
  for i in range(1,2**N):
    a.append(str1.format(i, fill='0'))

  sourcesInNode = []
  sourcesNotInNode = []
  subset = []
  sourceList = list(range(N))

  # find subset nodes of a node
  def node_subset(node, sourcesInNodes):
    return [node - 2**(i) for i in sourcesInNodes]

  # convert binary encoded string to integer list
  def string_to_integer_array(s, ch):
    N = len(s) 
    return [(N - i - 1) for i, ltr in enumerate(s) if ltr == ch]

  for j in range(len(a)):
  # index from right to left
    idxLR = string_to_integer_array(a[j],'1')
    sourcesInNode.append(idxLR)  
    sourcesNotInNode.append(list(set(sourceList) - set(idxLR)))
    subset.append(node_subset(j,idxLR))

  print("sources_and_subsets_nodes")
  return sourcesInNode, subset

def subset_to_indices(indices):
  print("subset_to_indeces")
  return [i for i in indices]

In [33]:
class Choquet_integral(torch.nn.Module):
  def __init__(self, N_in, N_out):
    super(Choquet_integral,self).__init__()
    self.N_in = N_in
    self.N_out = N_out
    self.nVars = 2**self.N_in - 2

    # The FM is initialized with mean
    dummy = (1./self.N_in) * torch.ones((self.nVars, self.N_out), requires_grad=True)

    # self.vars = torch.nn.Parameter( torch.Tensor(self.nVars,N_out))
    self.vars = torch.nn.Parameter(dummy)

    # following function uses numpy vs pytorch
    self.sourcesInNode, self.subset = sources_and_subsets_nodes(self.N_in)
    self.sourcesInNode = [torch.tensor(x) for x in self.sourcesInNode]
    self.subset = [torch.tensor(x) for x in self.subset]
    print("self.subset/n",self.subset)

  def forward(self,inputs):    
    self.FM = self.chi_nn_vars(self.vars)
    sortInputs, sortInd = torch.sort(inputs,1, True)
    M, N = inputs.size()
    sortInputs = torch.cat((sortInputs, torch.zeros(M,1)), 1)
    sortInputs = sortInputs[:,:-1] -  sortInputs[:,1:]
    out = torch.cumsum(torch.pow(2,sortInd),1) - torch.ones(1, dtype=torch.int64)
    data = torch.zeros((M,self.nVars+1))
    
    for i in range(M):
      data[i,out[i,:]] = sortInputs[i,:] 
      ChI = torch.matmul(data,self.FM)

    print("forward")
    return ChI

  # Converts NN-vars to FM vars
  def chi_nn_vars(self, chi_vars):
    # nVars,_ = chi_vars.size()
    chi_vars = torch.abs(chi_vars)
    # nInputs = inputs.get_shape().as_list()[1]
    FM = chi_vars[None, 0,:]
    
    for i in range(1,self.nVars):
      indices = subset_to_indices(self.subset[i])
      if (len(indices) == 1):
        FM = torch.cat((FM,chi_vars[None,i,:]),0)
      else:
      # ss=tf.gather_nd(variables, [[1],[2]])
        maxVal,_ = torch.max(FM[indices,:],0)
        temp = torch.add(maxVal,chi_vars[i,:])
    FM = torch.cat((FM,temp[None,:]),0)
    FM = torch.cat([FM, torch.ones((1,self.N_out))],0)
    FM = torch.min(FM, torch.ones(1))  

    print("chi_nn_vars")
    return FM

In [34]:
# training samples size
M = 10
# number of inputs
N_in = 3
# number of outputs aka number of Choquet integral neurons
N_out = 2  
# Create a synthetic dataset via random sampling from a normal distribution with mean =-1 and std=2
X_train = np.random.rand(M,N_in)*2-1
# Let's specify the FMs  (There will be N_out number of FMs)
# Herein we adopt binary encoding instead of lexicographic encoding to represent a FM that is easier to code. 
# As for example, an FM for three inputs using lexicographic encoding is, g = {g_1, g_2, g_3, g_{12}, g_{13}, g_{23}, g_{123}}.
# whereas its binary encoding is g = {g_1, g_2, g_{12}, g_3 g_{13}, g_{23}, g_{123}}.
# For simplicity, here we use OWA. 
print(X_train)

[[-0.80043495  0.99409497 -0.87933035]
 [ 0.03203445 -0.75139734  0.10212008]
 [-0.48022687 -0.92587854 -0.43459494]
 [ 0.20525812  0.32462943 -0.25628559]
 [-0.81124076  0.02333231 -0.59672836]
 [-0.22704916 -0.46661703  0.02615763]
 [ 0.7793833   0.24269765  0.86285959]
 [-0.4987119  -0.16805052  0.2830831 ]
 [ 0.27700974 -0.23412249  0.83571603]
 [-0.08596147 -0.1970641   0.08727986]]


In [35]:
OWA = np.array([[0.7, 0.2, 0.1], # this is soft-max
                    [0.1,0.2,0.7]])  # soft-min
# The FMs of the above OWAs in binary encoding
# FM = [[0.7, 0.7, 0.9, 0.7, 0.9, 0.9, 1.0].
#      [0.1, 0.1, 0.3, 0.1, 0.3, 0.3, 1.0]]
print('Actual/groundtruth FMs in binary encoding:')
print('FM1 = ', np.array([0.7, 0.7, 0.9, 0.7, 0.9, 0.9, 1.0]))
print('FM2 = ', np.array([0.1, 0.1, 0.3, 0.1, 0.3, 0.3, 1.0]))

Actual/groundtruth FMs in binary encoding:
FM1 =  [0.7 0.7 0.9 0.7 0.9 0.9 1. ]
FM2 =  [0.1 0.1 0.3 0.1 0.3 0.3 1. ]


In [36]:
# Generate the label or the groundtruth based on the provided FMs/OWAs. The labels are two dimentional
label_train = np.matmul(np.sort(X_train), np.fliplr(OWA).T)
    
# Now we want to recover the FMs from the training data and groundtruth
# First, build a Choquet integral neuron with N_in inputs and N_out outputs
net = Choquet_integral(N_in,N_out)
    
# set the optimization algorithms and paramters the learning
learning_rate = 0.3;
    
# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)   
    
num_epochs = 300;
    
# convert from numpy to torch tensor
X_train = torch.tensor(X_train,dtype=torch.float) #.to("cuda")
label_train = torch.tensor(label_train,dtype=torch.float) #.to("cuda")

sources_and_subsets_nodes
self.subset/n [tensor([-1]), tensor([-1]), tensor([0, 1]), tensor([-1]), tensor([0, 3]), tensor([1, 3]), tensor([2, 4, 5])]


In [37]:
print(X_train)

tensor([[-0.8004,  0.9941, -0.8793],
        [ 0.0320, -0.7514,  0.1021],
        [-0.4802, -0.9259, -0.4346],
        [ 0.2053,  0.3246, -0.2563],
        [-0.8112,  0.0233, -0.5967],
        [-0.2270, -0.4666,  0.0262],
        [ 0.7794,  0.2427,  0.8629],
        [-0.4987, -0.1681,  0.2831],
        [ 0.2770, -0.2341,  0.8357],
        [-0.0860, -0.1971,  0.0873]])


In [ ]:
y_pred = net(X_train)

In [ ]:
model.train()
# optimize
for t in range(num_epochs):
# Forward pass: Compute predicted y by passing x to the model
  y_pred = net(X_train)
# Compute the loss
  loss = criterion(y_pred, label_train)
# Zero gradients, perform a backward pass, and update the weights.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()  

In [ ]:
# Finally, the learned FMs
FM_learned = (net.chi_nn_vars(net.vars).cpu()).detach().numpy()
print('\n\nLearned FMs:')
print('FM1 = ', FM_learned[:,0])
print('FM2 = ',FM_learned[:,1])

In [2]:
!apt-get install google-perftools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
google-perftools is already the newest version (2.5-2.2ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [3]:
!echo "LD_PRELOAD=/usr/lib/libtcmalloc.so.4" | tee -a /etc/environment

LD_PRELOAD=/usr/lib/libtcmalloc.so.4


In [22]:
import numpy as np
# ========================================
# ~ MEASURES
# ========================================


def _differentiation_1_distance(X):
    #Perform differentiation for each consecuent point in the X dataset (time series)
    print("dis:")
    print(X[0], X[1:] - X[0:-1])
    return np.append(X[0], X[1:] - X[0:-1])


def generate_cardinality(N, p = 2):
    '''
    Generate the cardinality measure for a N-sized vector.
    '''
    print("FM:")
    for x in np.arange(N, 0, -1):
      print((x/ N)**p) 
    return [(x/ N)**p for x in np.arange(N, 0, -1)]


def generate_cardinality_matrix(N, matrix_shape, p = 2):
    '''
    Generate the cardinality measure for a N-sized vector, and returns it in a matrix shape.
    Use this if you cannot broadcast generate_cardinality() correctly.
    N and matrix_shape must be coherent (matrix_shape[0] == N)
    '''
    res = np.zeros(matrix_shape)
    dif_elements = [(x/ N)**p for x in np.arange(N, 0, -1)]

    for ix, elements in enumerate(dif_elements ):
        res[ix,...] = dif_elements[ix]

    return res
# ========================================
# ~ INTEGRALS
# ========================================

def choquet_integral_symmetric(X, measure=None, axis=0, keepdims=True):
    '''
    Aggregates a numpy array alongise an axis using the choquet integral.
    
    :param X: Data to aggregate.
    :param measure: Vector containing the measure numeric values (Symmetric!)
    :param axis: Axis alongside to aggregate.
    '''
    if measure is None:
        measure = generate_cardinality(
            X.shape[axis])

    X_sorted = np.sort(X, axis = axis)

    X_differenced = np.apply_along_axis(
    _differentiation_1_distance, axis, X_sorted)
    X_agg  = np.apply_along_axis(lambda a: np.dot(a, measure), axis, X_differenced)

    if keepdims:
        X_agg = np.expand_dims(X_agg, axis=axis)

    return X_agg



def sugeno_fuzzy_integral(X, measure=None, axis = 0, keepdims=True):
    '''
    Aggregates data using a generalization of the Choquet integral.
    
    :param X: Data to aggregate.
    :param measure: Vector containing the measure numeric values.
    :param axis: Axis alongside to aggregate.
    '''
    if measure is None:
        measure = generate_cardinality(
                X.shape[axis])

    return sugeno_fuzzy_integral_generalized(X, measure, axis, np.minimum, np.amax, keepdims)

In [28]:
choquet_integral_symmetric(np.array([0.2,0.9,0.8]))

FM:
1.0
0.4444444444444444
0.1111111111111111
dis:
0.2 [0.6 0.1]


array([0.47777778])

In [13]:
sugeno_fuzzy_integral(np.array([0.2,0.9,0.8]))

array([0.44444444])

In [18]:
np.array(([0.2,0.9,0.8],[0.4,0.5,0.7])).shape

(2, 3)

In [23]:
choquet_integral_symmetric(np.array(([0.2,0.9,0.8],[0.4,0.5,0.7])))

FM:
1.0
0.25
dis:
0.2 [0.2]
dis:
0.5 [0.4]
dis:
0.7 [0.1]


array([[0.25 , 0.6  , 0.725]])

In [27]:
choquet_integral_symmetric(np.array(([0.2,0.9,0.8],[0.4,0.5,0.7])),[0,1])

dis:
0.2 [0.2]
dis:
0.5 [0.4]
dis:
0.7 [0.1]


array([[0.2, 0.4, 0.1]])